# Analyze the effect of clustering on the speed data

<b>Intuition</b> : Classification is not possible to classify the speed range between 3.0mph-7.0mph as some people might be walking or running at intermittent speeds such as 4.0-6.0mph.

Therefore in this I have used different clustering algorithms. The challenge associated with this model is that we cannot test it against metrics such as homogeniety score or sihouette coeffecient. So a step in hyperparameter tuning involves using a RandomForestRegressor to rank the models based on the precision observed in the model.

## Algorithms used :
<ol>
    <li>KMeans</li>
    <li>DbScan</li>
    <li>Affinity Propagation</li>
</ol>

## KMeans